In [ ]:
import os, sys
from pathlib import Path

# Colab basics
print("Python:", sys.version)

# Keep installs small-ish. If you already ran this once, you can skip it.
!pip -q install pandas numpy pyarrow lightgbm scikit-learn requests

import pandas as pd
import numpy as np
import requests
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, average_precision_score

# Project folder so the Files pane doesn't turn into a mess
WORK = Path("/content/dsbios")
WORK.mkdir(exist_ok=True)
os.chdir(WORK)

print("Working dir:", WORK)

In [ ]:
import json
import re
from pathlib import Path
import pandas as pd
import numpy as np

FILE = Path("drugshortages.json")
if not FILE.exists():
    raise FileNotFoundError(
        "Missing drugshortages.json. Upload your OpenFDA download and rename it to 'drugshortages.json'."
    )

print("Found drugshortages.json (MB):", round(FILE.stat().st_size / 1e6, 2))

# Load JSON and figure out where the actual records live
with open(FILE, "r") as f:
    raw = json.load(f)

if isinstance(raw, dict):
    if "results" in raw:
        records = raw["results"]
    elif "data" in raw:
        records = raw["data"]
    else:
        raise RuntimeError(f"Don't recognize JSON keys: {list(raw.keys())[:30]}")
elif isinstance(raw, list):
    records = raw
else:
    raise RuntimeError(f"Unexpected JSON root type: {type(raw)}")

print("Records:", len(records))

# Pull only what we need
rows = []
for r in records:
    tc = r.get("therapeutic_category", "")
    if isinstance(tc, list):
        tc = ", ".join(tc)

    rows.append({
        "generic_name": r.get("generic_name", ""),
        "initial_posting_date": r.get("initial_posting_date", ""),
        "status": r.get("status", ""),
        "dosage_form": r.get("dosage_form", ""),
        "therapeutic_category": tc,
        "company_name": r.get("company_name", "")
    })

df = pd.DataFrame(rows)
print("Raw shape:", df.shape)

def normalize_drug(name: str) -> str:
    if not isinstance(name, str):
        return ""
    s = name.lower()
    s = re.sub(r"\(.*?\)", "", s)                 # strip parentheses junk
    s = re.sub(r"[^a-z0-9\s\-\/]", " ", s)        # keep basic tokens
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["drug_norm"] = df["generic_name"].apply(normalize_drug)

# Dates
df["start_date"] = pd.to_datetime(df["initial_posting_date"], errors="coerce")
df = df.dropna(subset=["start_date"]).copy()
df["start_year"] = df["start_date"].dt.year.astype(int)

# Count shortage starts per (drug, year)
shortage_events = (
    df.groupby(["drug_norm", "start_year"])
      .size()
      .reset_index(name="shortages_started")
)

shortage_events.to_csv("shortage_events_by_drug_year.csv", index=False)
print("Saved shortage_events_by_drug_year.csv (rows):", len(shortage_events))


In [ ]:
from pathlib import Path

REQUIRED = [
    "shortage_events_by_drug_year.csv",
    "orange_book.zip",
    "partd_spending_by_drug.csv"
]

missing = []
for f in REQUIRED:
    p = Path(f)
    if not p.exists():
        missing.append(f)

if missing:
    raise RuntimeError(
        "Missing files:\n  - " + "\n  - ".join(missing) +
        "\n\nUpload them with the exact names and rerun."
    )

print("All required inputs present.")


In [ ]:
import zipfile
from pathlib import Path

ORANGE_ZIP = Path("orange_book.zip")
if not ORANGE_ZIP.exists():
    raise FileNotFoundError("orange_book.zip not found.")

out_dir = Path("orange_book_unzipped")
out_dir.mkdir(exist_ok=True)

with zipfile.ZipFile(ORANGE_ZIP, "r") as z:
    z.extractall(out_dir)

# Find Products*.txt (Orange Book naming varies a bit)
products_candidates = [p for p in out_dir.glob("*") if p.name.lower().startswith("products")]
if not products_candidates:
    raise RuntimeError("Couldn't find a Products file after unzip. Check what's inside the zip.")

PRODUCTS_TXT = products_candidates[0]
print("Using Products file:", PRODUCTS_TXT.name)


In [ ]:
import pandas as pd
import re
from pathlib import Path

out_dir = Path("orange_book_unzipped")
products_candidates = [p for p in out_dir.glob("*") if p.name.lower().startswith("products")]
if not products_candidates:
    raise RuntimeError("Products file not found. Rerun the unzip cell.")
PRODUCTS_TXT = products_candidates[0]

# Orange Book Products is usually "~" delimited
products = pd.read_csv(PRODUCTS_TXT, sep="~", dtype=str, engine="python")

def find_col(df, keywords):
    for k in keywords:
        for c in df.columns:
            if k in c.lower():
                return c
    return None

ingredient_col = find_col(products, ["ingredient"])
applicant_col  = find_col(products, ["applicant"])

if ingredient_col is None or applicant_col is None:
    raise RuntimeError(
        "Couldn't auto-find ingredient/applicant columns.\n"
        f"Columns: {list(products.columns)}"
    )

def normalize_drug(name: str) -> str:
    if not isinstance(name, str):
        return ""
    s = name.lower()
    s = re.sub(r"\(.*?\)", "", s)
    s = re.sub(r"[^a-z0-9\s\-\/]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

ob = products[[ingredient_col, applicant_col]].copy()
ob.columns = ["ingredient", "applicant"]

ob["drug_norm"] = ob["ingredient"].apply(normalize_drug)
ob["applicant_norm"] = ob["applicant"].fillna("").astype(str).str.lower().str.strip()

manufacturer_counts = (
    ob.groupby("drug_norm")["applicant_norm"]
      .nunique()
      .reset_index(name="num_manufacturers")
)

manufacturer_counts["num_manufacturers"] = manufacturer_counts["num_manufacturers"].fillna(0).astype(int)
manufacturer_counts.to_csv("manufacturer_counts_by_drug.csv", index=False)

print("Saved manufacturer_counts_by_drug.csv (rows):", len(manufacturer_counts))


In [ ]:
import pandas as pd
from pathlib import Path

PARTD = Path("partd_spending_by_drug.csv")
if not PARTD.exists():
    raise FileNotFoundError("partd_spending_by_drug.csv not found.")

cols = pd.read_csv(PARTD, nrows=0).columns.tolist()
print("Columns:", cols)
print("\nSample:")
print(pd.read_csv(PARTD, nrows=5, dtype=str))


In [ ]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

PARTD = Path("partd_spending_by_drug.csv")
df = pd.read_csv(PARTD, dtype=str)

needed = ["Gnrc_Name", "Mftr_Name"]
for c in needed:
    if c not in df.columns:
        raise RuntimeError(f"Missing '{c}'. Found columns: {list(df.columns)}")

claim_cols = [c for c in df.columns if c.startswith("Tot_Clms_")]
if not claim_cols:
    raise RuntimeError("No Tot_Clms_YYYY columns found. Check the file format.")

# Avoid double counting: keep only the "Overall" row per drug
overall_mask = df["Mftr_Name"].astype(str).str.strip().str.lower() == "overall"
df_overall = df[overall_mask].copy()
if df_overall.empty:
    raise RuntimeError("Couldn't find rows where Mftr_Name == 'Overall'. Check Mftr_Name values.")

def normalize_drug(name: str) -> str:
    if not isinstance(name, str):
        return ""
    s = name.lower()
    s = re.sub(r"\(.*?\)", "", s)
    s = re.sub(r"[^a-z0-9\s\-\/]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df_overall["drug_norm"] = df_overall["Gnrc_Name"].fillna("").astype(str).apply(normalize_drug)

rows = []
for col in claim_cols:
    # expecting Tot_Clms_2019, etc.
    try:
        year = int(col.split("_")[-1])
    except ValueError:
        continue

    tmp = df_overall[["drug_norm", col]].copy()
    tmp = tmp.rename(columns={col: "claims"})
    tmp["claims"] = pd.to_numeric(tmp["claims"].astype(str).str.replace(",", ""), errors="coerce").fillna(0)
    tmp["year"] = year
    rows.append(tmp)

demand = pd.concat(rows, ignore_index=True)
demand = demand.groupby(["drug_norm", "year"], as_index=False)["claims"].sum()

demand.to_csv("demand_by_drug_year.csv", index=False)
print("Saved demand_by_drug_year.csv (rows):", len(demand), "| years:", sorted(demand["year"].unique()))


In [ ]:
import pandas as pd
import re

FORM_WORDS = set("""
tablet tablets tab capsule capsules cap solution solutions soln injection injections inj
suspension suspensions susp syrup ointment cream gel spray patch patches aerosol foam
extended release er xr dr sr delayed release immediate release
iv intravenous im intramuscular subcutaneous sc oral rectal topical ophthalmic otic nasal
""".split())

def drug_key(name: str) -> str:
    if not isinstance(name, str):
        return ""
    s = name.lower()
    s = re.sub(r"\(.*?\)", " ", s)
    s = s.replace("%", " ")
    s = re.sub(r"[^a-z0-9\s\-\/]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()

    toks = []
    for t in s.split():
        if t.isdigit():
            continue
        if re.fullmatch(r"\d+(\.\d+)?", t):
            continue
        if t in FORM_WORDS:
            continue
        toks.append(t)

    return " ".join(sorted(set(toks)))

# Add drug_key into all three datasets (in-place)
demand = pd.read_csv("demand_by_drug_year.csv")
mfg    = pd.read_csv("manufacturer_counts_by_drug.csv")
labels = pd.read_csv("shortage_events_by_drug_year.csv")

for name, df in [("demand", demand), ("mfg", mfg), ("labels", labels)]:
    if "drug_norm" not in df.columns:
        raise RuntimeError(f"{name} is missing drug_norm. Columns: {list(df.columns)}")

demand["drug_key"] = demand["drug_norm"].apply(drug_key)
mfg["drug_key"]    = mfg["drug_norm"].apply(drug_key)
labels["drug_key"] = labels["drug_norm"].apply(drug_key)

demand.to_csv("demand_by_drug_year.csv", index=False)
mfg.to_csv("manufacturer_counts_by_drug.csv", index=False)
labels.to_csv("shortage_events_by_drug_year.csv", index=False)

print("Added drug_key to demand/mfg/labels and saved.")


In [ ]:
import pandas as pd
import numpy as np

demand = pd.read_csv("demand_by_drug_year.csv")
mfg    = pd.read_csv("manufacturer_counts_by_drug.csv")
labels = pd.read_csv("shortage_events_by_drug_year.csv")

# labels might have start_year depending on where it came from
labels2 = labels.copy()
if "year" in labels2.columns:
    labels2["year"] = pd.to_numeric(labels2["year"], errors="coerce")
elif "start_year" in labels2.columns:
    labels2 = labels2.rename(columns={"start_year": "year"})
    labels2["year"] = pd.to_numeric(labels2["year"], errors="coerce")
else:
    raise RuntimeError(f"Labels file missing year/start_year. Columns: {list(labels2.columns)}")

labels2 = labels2.dropna(subset=["year"]).copy()
labels2["year"] = labels2["year"].astype(int)

if "shortages_started" not in labels2.columns:
    raise RuntimeError(f"Labels file missing shortages_started. Columns: {list(labels2.columns)}")

# Set of (drug_key, year) where a shortage starts
shortage_started_set = set(zip(labels2["drug_key"], labels2["year"]))

# Join demand + manufacturer count
df = demand.merge(mfg[["drug_key", "num_manufacturers"]], on="drug_key", how="left")
df["num_manufacturers"] = df["num_manufacturers"].fillna(0).astype(int)

# Prior shortage totals by drug
hist = labels2.groupby("drug_key")["shortages_started"].sum().to_dict()
df["prior_shortage_total"] = df["drug_key"].map(hist).fillna(0).astype(int)

# Label: does a shortage start next year?
df["label_next_year"] = df.apply(
    lambda r: 1 if (r["drug_key"], int(r["year"]) + 1) in shortage_started_set else 0,
    axis=1
)

# Demand growth features
df = df.sort_values(["drug_key", "year"])
df["claims_prev"] = df.groupby("drug_key")["claims"].shift(1)
df["claims_growth_pct"] = (df["claims"] - df["claims_prev"]) / df["claims_prev"]
df["claims_growth_pct"] = df["claims_growth_pct"].replace([np.inf, -np.inf], np.nan).fillna(0)

df.to_csv("model_table.csv", index=False)
print("Saved model_table.csv (rows):", len(df), "| label counts:", df["label_next_year"].value_counts().to_dict())


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, average_precision_score

df = pd.read_csv("model_table.csv")

feature_cols = ["claims", "claims_growth_pct", "num_manufacturers", "prior_shortage_total"]
X = df[feature_cols]
y = df["label_next_year"].astype(int)

years = sorted(df["year"].unique())
if len(years) < 3:
    raise RuntimeError(f"Need at least 3 years for train/val/test split. Years found: {years}")

train_years = years[:-2]
val_year = years[-2]
test_year = years[-1]

train_idx = df["year"].isin(train_years)
val_idx = df["year"] == val_year
test_idx = df["year"] == test_year

X_train, y_train = X[train_idx], y[train_idx]
X_val, y_val     = X[val_idx], y[val_idx]
X_test, y_test   = X[test_idx], y[test_idx]

neg = max((y_train == 0).sum(), 1)
pos = max((y_train == 1).sum(), 1)
scale_pos_weight = neg / pos

train_set = lgb.Dataset(X_train, label=y_train)
val_set   = lgb.Dataset(X_val, label=y_val, reference=train_set)

params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "max_depth": 8,
    "min_data_in_leaf": 20,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.9,
    "bagging_freq": 1,
    "scale_pos_weight": scale_pos_weight,
    "verbosity": -1,
    "seed": 42,
}

model = lgb.train(
    params,
    train_set,
    num_boost_round=2000,
    valid_sets=[val_set],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=100),
    ],
)

pred_test = model.predict(X_test, num_iteration=model.best_iteration)

# Metrics only make sense if test has both classes
if len(np.unique(y_test)) > 1:
    auc = roc_auc_score(y_test, pred_test)
    ap = average_precision_score(y_test, pred_test)
    print("Test AUC:", round(auc, 4), "| Test AP:", round(ap, 4))
else:
    print("Test year has a single class; skipping AUC/AP.")

# Precision@K (how many of the top-K are real positives)
test_df = df.loc[test_idx].copy()
test_df["risk_score"] = pred_test
test_df = test_df.sort_values("risk_score", ascending=False)

for k in [10, 20, 50]:
    if len(test_df) >= k:
        print(f"Precision@{k}:", round(test_df.head(k)["label_next_year"].mean(), 3))

# Score everything + make a watchlist for latest year
df["risk_score"] = model.predict(X, num_iteration=model.best_iteration)

latest_year = int(df["year"].max())
watch = df[df["year"] == latest_year].sort_values("risk_score", ascending=False).head(50)

watch.to_csv("watchlist_top50.csv", index=False)
print("Saved watchlist_top50.csv (latest year):", latest_year)

imp = pd.DataFrame({
    "feature": feature_cols,
    "gain": model.feature_importance(importance_type="gain")
}).sort_values("gain", ascending=False)

imp.to_csv("feature_importance.csv", index=False)
model.save_model("dsbios_lgb_model.txt")

print("Saved feature_importance.csv and dsbios_lgb_model.txt")


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

WATCH = Path("watchlist_top50.csv")
if not WATCH.exists():
    raise FileNotFoundError("watchlist_top50.csv not found. Train cell should create it.")

watch = pd.read_csv(WATCH).sort_values("risk_score", ascending=False).reset_index(drop=True)

watch["risk_rank"] = np.arange(1, len(watch) + 1)
if len(watch) > 1:
    watch["risk_percentile"] = (1 - (watch["risk_rank"] - 1) / (len(watch) - 1)) * 100
else:
    watch["risk_percentile"] = 100.0
watch["risk_percentile"] = watch["risk_percentile"].round(1)

def tier(p):
    if p >= 99: return "CRITICAL (Top 1%)"
    if p >= 95: return "HIGH (Top 5%)"
    if p >= 80: return "ELEVATED (Top 20%)"
    return "MONITOR"

watch["risk_tier"] = watch["risk_percentile"].apply(tier)

def explain(row):
    reasons = []
    if row.get("prior_shortage_total", 0) >= 2:
        reasons.append("repeat shortage history")
    if row.get("num_manufacturers", 0) != 0 and row.get("num_manufacturers", 999) <= 2:
        reasons.append("few manufacturers")
    if row.get("claims_growth_pct", 0) > 0.15:
        reasons.append("demand rising fast")
    return ", ".join(reasons) if reasons else "mixed signals (review)"

watch["top_reasons"] = watch.apply(explain, axis=1)

top20 = watch.head(20).copy()
presentable = top20[[
    "drug_key", "year", "risk_percentile", "risk_tier",
    "num_manufacturers", "claims", "claims_growth_pct",
    "prior_shortage_total", "top_reasons"
]].copy()

presentable.rename(columns={
    "drug_key": "Drug (Canonical)",
    "year": "Prediction Year",
    "risk_percentile": "Risk Percentile",
    "risk_tier": "Risk Tier",
    "num_manufacturers": "# Manufacturers (Orange Book)",
    "claims": "Medicare Part D Claims",
    "claims_growth_pct": "Claims Growth %",
    "prior_shortage_total": "Prior Shortages (OpenFDA)",
    "top_reasons": "Top Reasons"
}, inplace=True)

presentable.to_csv("top20_submission_table_presentable.csv", index=False)
watch.to_csv("watchlist_top50_presentable.csv", index=False)

print("Saved top20_submission_table_presentable.csv and watchlist_top50_presentable.csv")
print(presentable.to_string(index=False))
